In [ ]:
import obspy
import glob
import matplotlib.pyplot as plt
import os
import shutil
import subprocess

# Set up all paths that will be used
homedir = os.path.expanduser('~')
softwaredir = os.path.join(homedir, 'Desktop', 'software')
WINSUDSPATH = os.path.join(softwaredir,'winsuds','bin')
datatop = os.path.join(homedir, 'Desktop', 'DATA')
datadir = os.path.join(datatop, 'NevadoDelRuiz','suds2')
demux = os.path.join(WINSUDSPATH, 'demux.exe')
irig = os.path.join(WINSUDSPATH, 'irig.exe')
sud2sac = os.path.join(WINSUDSPATH, 'sud2sac.exe')
sud2msed = os.path.join(WINSUDSPATH, 'sud2msed.exe')

def runCommand(cmdlist):
    print(' '.join(cmdlist))
    result = subprocess.run(cmdlist, capture_output=True, text=True)
    print(result.stdout)
    
#### MAIN PROGRAM STARTS HERE #############
# Loop over year directories
yyyydirs = sorted(glob.glob(os.path.join(datadir, '[1-2][0-9][0-9][0-9]')))
for yyyydir in yyyydirs:
    #yyyy = os.path.basename(yyyydir)
    mmdirs = sorted(glob.glob(os.path.join(yyyydir,'[0-1][0-9]')))
    for mmdir in mmdirs:
        #mm = os.path.basename(mmdir)
        dddirs = sorted(glob.glob(os.path.join(mmdir, '[0-3][0-9]')))

        for dddir in dddirs:
            WVMfiles = sorted(glob.glob(os.path.join(dddir, '*.WVR')))
            print('Found %d WVM files in %s' % (len(WVMfiles), dddir))
            for WVMfile in WVMfiles:
                SUDSdirbase = WVMfile[0:-4]
                SUDSbasename = os.path.basename(SUDSdirbase)
                MSEEDfile = SUDSdirbase + '.mseed' # produced by recombining SAC file
                if os.path.isfile(MSEEDfile):
                    continue
                DMXfile = SUDSdirbase + '.dmx'
                print('Demultiplexing ' + WVMfile)
                runCommand([demux, WVMfile])
                if (os.path.exists(DMXfile)):
                    print('- Success')
                else:
                    print('Demultiplexing seems to have failed as ' + DMXfile + ' not created. Adding to list of bad WVM files.')
                    fbad = open('badWVMfileList.txt','a+')
                    fbad.write(WVMfile + '\n')
                    fbad.close()
                    continue

                print('Time correcting ' + DMXfile)
                runCommand([irig, DMXfile])

                # This produces one file per trace
                print('Converting ' + DMXfile + ' to SAC files')
                try:
                    runCommand([sud2sac, DMXfile])
                    os.remove(DMXfile)
                except:
                    print('Crashed on converting')
                SACdirbase = SUDSdirbase + '.sac' # produced  by sud2sac.exeyy
                sacfilelist = glob.glob(SACdirbase + '-???')

                # Now merge the SAC files into a single valid Miniseed file    
                if len(sacfilelist) > 0:
                    st = obspy.Stream()
                    for sacfile in sacfilelist:
                        print('Combining ' + sacfile + ' into Miniseed file ' + MSEEDfile)
                        try:
                            sacst = obspy.read(sacfile)
                            st = st + sacst
                        except:
                            pass
                        os.remove(sacfile)
                    if len(st) == 0:
                        print('No good SAC files')
                        continue
                    st.write(MSEEDfile)

                else:
                    print('No SAC files found matching ' + SACdirbase + '*. Must have been bad DMX file ' + DMXfile)
                    fbad = open('badDMXfileList.txt','a+')
                    fbad.write(DMXfile + '\n')
                    fbad.close()